<a href="https://colab.research.google.com/github/machiwao/python-custom-file-transfer-protocol/blob/main/OPSYS_Custom_File_Transfer_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompt: Networking: Develop a simple file transfer protocol using sockets in Python allowing users to transfer files between two computers over a network and handling connection issues

To create a simple file transfer protocol using sockets in Python, you can build both a server and a client script to send files between two computers. The server will listen for incoming connections and handle file reception, while the client will connect to the server and send a file.

Here’s an implementation that handles basic connection issues as well.

# Server Side

In [ ]:
import socket
import os

# Server-side parameters
SERVER_HOST = '0.0.0.0'  # Listen on all network interfaces
SERVER_PORT = 5001        # Arbitrary non-privileged port
BUFFER_SIZE = 4096        # Size of the data chunk to be received
SEPARATOR = "<SEPARATOR>" # Separator to distinguish between filename and file data

def start_server():
    # Create server socket (TCP)
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((SERVER_HOST, SERVER_PORT))
    server_socket.listen(5)
    print(f"[*] Listening on {SERVER_HOST}:{SERVER_PORT}")

    # Accept incoming connection
    client_socket, client_address = server_socket.accept()
    print(f"[+] {client_address} connected.")

    try:
        # Receive the file information
        received = client_socket.recv(BUFFER_SIZE).decode()
        filename, filesize = received.split(SEPARATOR)
        filename = os.path.basename(filename)
        filesize = int(filesize)

        print(f"[*] Receiving file: {filename}, Size: {filesize} bytes")

        # Open file for writing
        with open(filename, "wb") as file:
            total_received = 0
            while total_received < filesize:
                bytes_read = client_socket.recv(BUFFER_SIZE)
                if not bytes_read:
                    # If no more data, break the loop
                    break
                file.write(bytes_read)
                total_received += len(bytes_read)
                print(f"[*] Received {total_received}/{filesize} bytes")

        print(f"[+] File {filename} received successfully.")
    except Exception as e:
        print(f"[!] Error during file transfer: {e}")
    finally:
        client_socket.close()
        server_socket.close()
        print("[*] Connection closed.")

if __name__ == "__main__":
    start_server()

# Client Side

In [ ]:
import socket
import os

# Client-side parameters
SERVER_HOST = '127.0.0.1'  # Replace with the server's IP address
SERVER_PORT = 5001         # Same as the server
BUFFER_SIZE = 4096
SEPARATOR = "<SEPARATOR>"

def send_file(filename):
    # Get the file size
    filesize = os.path.getsize(filename)

    # Create client socket (TCP)
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    print(f"[*] Connecting to {SERVER_HOST}:{SERVER_PORT}")

    try:
        # Connect to the server
        client_socket.connect((SERVER_HOST, SERVER_PORT))
        print(f"[+] Connected to {SERVER_HOST}:{SERVER_PORT}")

        # Send filename and filesize to the server
        client_socket.send(f"{filename}{SEPARATOR}{filesize}".encode())

        # Send the file in chunks
        with open(filename, "rb") as file:
            while True:
                # Read the bytes from the file
                bytes_read = file.read(BUFFER_SIZE)
                if not bytes_read:
                    # File transmission is done
                    break
                # Send the chunk to the server
                client_socket.sendall(bytes_read)

        print(f"[+] File {filename} sent successfully.")
    except Exception as e:
        print(f"[!] Error during file transfer: {e}")
    finally:
        client_socket.close()
        print("[*] Connection closed.")

if __name__ == "__main__":
    filename = "path_to_your_file.txt"  # Replace with the actual file path
    send_file(filename)


Explanation:

Server:

Listens on a specified host and port.

Accepts connections from clients.

Receives the file's name and size, followed by the file data in chunks.

Saves the received file to the server's local directory.

Client:

Connects to the server.

Sends the file's name and size first, followed by the actual file in chunks.

Automatically reads the file and transmits it until done.


Handling Connection Issues:

Timeouts: You can set timeouts for socket operations using socket.settimeout(time_in_seconds).

Error Handling: Try-except blocks are used to catch connection issues such as refusal, disconnections, or file transmission errors.

Graceful Closing: Sockets are closed properly whether the transfer was successful or failed.

This solution enables simple file transfer between two computers. You can test it by running the server on one machine and the client on another, making sure they are on the same network.